# NIPS 2017 Figure 3

In [ ]:
%run -i NIPS2017.ipynb

# FIGURE and GRID
FIG_HEIGHT_MM = 90
FIG_WIDTH_MM = FIG_WIDTH_MM  # set in NIPS2017 notebook to a default value for all figures
FIG_N_ROWS = 3
ROW_1_NCOLS = 3
ROW_1_HEIGHT_MM = FIG_HEIGHT_MM / FIG_N_ROWS
ROW_1_WIDTH_COL_1_MM = FIG_WIDTH_MM / ROW_1_NCOLS
ROW_1_WIDTH_COL_2_MM = 0.8*FIG_WIDTH_MM / ROW_1_NCOLS
ROW_1_WIDTH_COL_3_MM = 1.2*FIG_WIDTH_MM / ROW_1_NCOLS
ROW_2_NCOLS = 3
ROW_2_HEIGHT_MM = FIG_HEIGHT_MM / FIG_N_ROWS
ROW_2_WIDTH_COL_1_MM = FIG_WIDTH_MM / ROW_2_NCOLS
ROW_2_WIDTH_COL_2_MM = 0.8*FIG_WIDTH_MM / ROW_2_NCOLS
ROW_2_WIDTH_COL_3_MM = 1.2*FIG_WIDTH_MM / ROW_2_NCOLS
ROW_3_NCOLS = 1
ROW_3_HEIGHT_MM = FIG_HEIGHT_MM / FIG_N_ROWS
ROW_3_WIDTH_COL_1_MM = FIG_WIDTH_MM / ROW_3_NCOLS

# PATHS
PANEL_A = PATH_DROPBOX_FIGS + 'fig3_a.svg'
PANEL_B = PATH_DROPBOX_FIGS + 'fig3_b.svg'
PANEL_C = PATH_DROPBOX_FIGS + 'fig3_c.svg'
PANEL_D = PATH_DROPBOX_FIGS + 'fig3_d.svg'
PANEL_E = PATH_DROPBOX_FIGS + 'fig3_e.svg'
PANEL_F = PATH_DROPBOX_FIGS + 'fig3_f.svg'
PANEL_G = PATH_DROPBOX_FIGS + 'fig3_g.svg'

## Row 1

Load data for row 1

In [ ]:
prefix = 'hh_1000_iwloss_svi_cyth_seed1_v3pa01'

dists, infos, losses, nets, posteriors, sims = io.load_prefix(dirs['dir_nets_hh'], prefix)
sim = io.last(sims)
obs_stats = sim.obs
y_obs = sim.obs_trace.reshape(-1,1)
gt = sim.true_params
labels_params = sim.labels_params
param_invtransform = sim.param_invtransform
param_transform = sim.param_transform
gt_transf = param_transform(gt)
n_params = sim.n_params
prior_min = sim.prior_min
prior_max = sim.prior_max
prior_lims = np.concatenate((prior_min.reshape(-1,1),prior_max.reshape(-1,1)),axis=1)
t = sim.t
I = sim.I_obs
A_soma = sim.A_soma
dt = sim.dt
duration = np.max(t)
posterior = io.nth(posteriors, -1)
bm = sim.bm
init = sim.init

m, S = posterior.calc_mean_and_cov()
diff_params = np.divide(param_invtransform(m)-gt,gt)
err_params_up_norm = np.divide( (param_invtransform(m+np.sqrt(np.diag(S))) - param_invtransform(m)) , gt )
err_params_down_norm = np.divide( param_invtransform(m) - param_invtransform(m-np.sqrt(np.diag(S))) , gt )

### Panel A

In [ ]:
# matplotlib takes figsize specified as inches
# in NIPS2016 SCALE_IN and the conversion function are defined
fig_inches = (SCALE_IN*mm2inches(ROW_1_WIDTH_COL_1_MM), SCALE_IN*mm2inches(0.95*ROW_1_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=fig_inches)

    gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[4, 1])
    ax = plt.subplot(gs[0])
    plt.plot(t, y_obs, color = COL['GT'], lw=2, label='')
    plt.ylabel('voltage (mV)')
    plt.legend(bbox_to_anchor=(1.15, 1), loc='upper right')

    ax.set_xticks([])
    ax.set_yticks([-80, -20, 40])

    ax = plt.subplot(gs[1])
    plt.plot(t,I*A_soma*1e3,'r', lw=2)
    plt.xlabel('time (ms)')
    plt.ylabel('input (nA)')

    ax.set_xticks([0, duration/2, duration])
    ax.set_yticks([0, 1.1*np.max(I*A_soma*1e3)])
    ax.yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.2f'))

    plt.savefig(PANEL_A, facecolor='None', transparent=True)  # the figure is saved as svg
    plt.close()

svg(PANEL_A)

panel = st.fromfile(PANEL_A)
width, height = panel.get_size()
print('size of svg : {}'.format((width, height)))
print('aspect ratio : {}'.format(get_num(width) / get_num(height)))
print('desired ratio : {}'.format(ROW_1_WIDTH_COL_1_MM / ROW_1_HEIGHT_MM))

### Panel B

In [ ]:
# matplotlib takes figsize specified as inches
# in NIPS2016 SCALE_IN and the conversion function are defined
fig_inches = (SCALE_IN*mm2inches(0.8*ROW_1_WIDTH_COL_2_MM), SCALE_IN*mm2inches(ROW_1_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC):
    plot_pdf(posterior, lims=prior_lims, gt=gt_transf, labels_params=labels_params, partial=True,
             figsize=fig_inches,fontscale=0.6)
    plt.savefig(PANEL_B, facecolor='None', transparent=True)  # the figure is saved as svg
    plt.close()
    
svg(PANEL_B)

panel = st.fromfile(PANEL_B)
width, height = panel.get_size()
print('size of svg : {}'.format((width, height)))
print('aspect ratio : {}'.format(get_num(width) / get_num(height)))
#print('desired ratio : {}'.format(ROW_1_WIDTH_COL_1_MM / ROW_1_HEIGHT_MM))

### Panel C

In [ ]:
# matplotlib takes figsize specified as inches
# in NIPS2016 SCALE_IN and the conversion function are defined
fig_inches = (SCALE_IN*mm2inches(ROW_1_WIDTH_COL_3_MM), SCALE_IN*mm2inches(ROW_1_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=fig_inches)

    num_samp = 3
    num_colors = num_samp+1
    cm1 = mpl.cm.Oranges
    col1 = [cm1(1.*i/num_colors) for i in range(num_colors)]

    # sampling at contour of 1 covariance away from mean
    x_samp = np.random.randn(n_params,num_samp)
    x_samp = np.divide(x_samp,np.linalg.norm(x_samp,axis=0))
    x_samp = (np.dot(S,x_samp)).T+m

    params = np.concatenate((np.array([param_invtransform(m)]),param_invtransform(x_samp)))

    V = np.zeros((len(t),1+num_samp))
    for i in range(1+num_samp):
        hh = bm.HH(init, params[i,:].reshape(1,-1),seed=230+i)
        V[:,i] = hh.sim_time(dt, t, I)[:,0]
        if i>0:
            plt.plot(t, V[:, i], color = col1[i-1], lw=2, label='sample '+str(num_samp-i+1))

    # plotting simulation
    plt.plot(t, V[:, 0], color=COL['SNPE'], lw=2, label='mode')
    plt.plot(t, y_obs, color=COL['GT'], lw=2, label='observation')
    plt.xlabel('time (ms)')
    plt.ylabel('voltage (mV)')

    ax = plt.gca()
    #handles, labels = ax.get_legend_handles_labels()
    #ax.legend(handles[::-1], labels[::-1], bbox_to_anchor=(1.3, 1), loc='upper right')

    ax.set_xticks([0, duration/2, duration])
    ax.set_yticks([-80, -20, 40])

    plt.savefig(PANEL_C, facecolor=plt.gcf().get_facecolor(), transparent=True)  # the figure is saved as svg
    plt.close()

svg(PANEL_C)

panel = st.fromfile(PANEL_C)
width, height = panel.get_size()
print('size of svg : {}'.format((width, height)))
print('aspect ratio : {}'.format(get_num(width) / get_num(height)))
print('desired ratio : {}'.format(ROW_1_WIDTH_COL_1_MM / ROW_1_HEIGHT_MM))

## Row 2

Load data for row 2

In [ ]:
prefix_realdata = 'hh_1000_iwloss_svi_cyth_realdata_seed1_v3pa01'  # ours, real data

dists, infos, losses, nets, posteriors, sims = io.load_prefix(dirs['dir_nets_hh'], prefix_realdata)
sim = io.last(sims)
obs_stats = sim.obs
y_obs = sim.obs_trace.reshape(-1,1)
gt = sim.true_params
labels_params = sim.labels_params
param_invtransform = sim.param_invtransform
param_transform = sim.param_transform
gt_transf = param_transform(gt)
n_params = sim.n_params
prior_min = sim.prior_min
prior_max = sim.prior_max
prior_lims = np.concatenate((prior_min.reshape(-1,1),prior_max.reshape(-1,1)),axis=1)
t = sim.t
I = sim.I_obs
A_soma = sim.A_soma
dt = sim.dt
duration = np.max(t)
posterior = io.last(posteriors)
bm = sim.bm
init = sim.init

m, S = posterior.calc_mean_and_cov()
diff_params = np.divide(param_invtransform(m)-gt,gt)
err_params_up_norm = np.divide( (param_invtransform(m+np.sqrt(np.diag(S))) - param_invtransform(m)) , gt )
err_params_down_norm = np.divide( param_invtransform(m) - param_invtransform(m-np.sqrt(np.diag(S))) , gt )

### Panel D

In [ ]:
# matplotlib takes figsize specified as inches
# in NIPS2016 SCALE_IN and the conversion function are defined
fig_inches = (SCALE_IN*mm2inches(ROW_2_WIDTH_COL_1_MM), SCALE_IN*mm2inches(0.95*ROW_2_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=fig_inches)

    gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[4, 1])
    ax = plt.subplot(gs[0])
    plt.plot(t, y_obs, color = COL['GT'], lw=2, label='')
    plt.ylabel('voltage (mV)')
    plt.legend(bbox_to_anchor=(1.15, 1), loc='upper right')

    ax.set_xticks([])
    ax.set_yticks([-80, -20, 40])

    ax = plt.subplot(gs[1])
    plt.plot(t,I*A_soma*1e3,'r', lw=2)
    plt.xlabel('time (ms)')
    plt.ylabel('input (nA)')

    ax.set_xticks([0, duration/2, duration])
    ax.set_yticks([0, 1.1*np.max(I*A_soma*1e3)])
    ax.yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.2f'))

    plt.savefig(PANEL_D, facecolor='None', transparent=True)  # the figure is saved as svg
    plt.close()

svg(PANEL_D)

panel = st.fromfile(PANEL_D)
width, height = panel.get_size()
print('size of svg : {}'.format((width, height)))
print('aspect ratio : {}'.format(get_num(width) / get_num(height)))
print('desired ratio : {}'.format(ROW_1_WIDTH_COL_1_MM / ROW_1_HEIGHT_MM))

### Panel E

In [ ]:
# matplotlib takes figsize specified as inches
# in NIPS2016 SCALE_IN and the conversion function are defined
fig_inches = (SCALE_IN*mm2inches(0.8*ROW_2_WIDTH_COL_2_MM), SCALE_IN*mm2inches(ROW_2_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=fig_inches)

    plot_pdf(posterior, lims=prior_lims, labels_params=labels_params, partial=True,
             figsize=fig_inches,fontscale=0.6)

    plt.savefig(PANEL_E, facecolor='None', transparent=True)  # the figure is saved as svg
    plt.close()
    
svg(PANEL_E)

panel = st.fromfile(PANEL_E)
width, height = panel.get_size()
print('size of svg : {}'.format((width, height)))
print('aspect ratio : {}'.format(get_num(width) / get_num(height)))
print('desired ratio : {}'.format(ROW_2_WIDTH_COL_2_MM / ROW_2_HEIGHT_MM))

### Panel F

In [ ]:
# matplotlib takes figsize specified as inches
# in NIPS2016 SCALE_IN and the conversion function are defined
fig_inches = (SCALE_IN*mm2inches(ROW_2_WIDTH_COL_3_MM), SCALE_IN*mm2inches(ROW_2_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=fig_inches)

    num_samp = 3
    num_colors = num_samp+1
    cm1 = mpl.cm.Oranges
    col1 = [cm1(1.*i/num_colors) for i in range(num_colors)]

    # sampling at contour of 1 covariance away from mean
    x_samp = np.random.randn(n_params,num_samp)
    x_samp = np.divide(x_samp,np.linalg.norm(x_samp,axis=0))
    x_samp = (np.dot(S,x_samp)).T+m

    params = np.concatenate((np.array([param_invtransform(m)]),param_invtransform(x_samp)))

    V = np.zeros((len(t),1+num_samp))
    for i in range(1+num_samp):
        hh = bm.HH(init, params[i,:].reshape(1,-1),seed=230+i)
        V[:,i] = hh.sim_time(dt, t, I)[:,0]
        if i>0:
            plt.plot(t, V[:, i], color = col1[i-1], lw=2, label='sample '+str(num_samp-i+1))

    # plotting simulation
    plt.plot(t, V[:, 0], color=COL['SNPE'], lw=2, label='mode')
    plt.plot(t, y_obs, color=COL['GT'], lw=2, label='original')
    plt.xlabel('time (ms)')
    plt.ylabel('voltage (mV)')

    ax = plt.gca()
    #handles, labels = ax.get_legend_handles_labels()
    #ax.legend(handles[::-1], labels[::-1], bbox_to_anchor=(1.3, 1), loc='upper right')

    ax.set_xticks([0, duration/2, duration])
    ax.set_yticks([-80, -20, 40])

    plt.savefig(PANEL_F, facecolor=plt.gcf().get_facecolor(), transparent=True)  # the figure is saved as svg
    plt.close()

svg(PANEL_F)

panel = st.fromfile(PANEL_F)
width, height = panel.get_size()
print('size of svg : {}'.format((width, height)))
print('aspect ratio : {}'.format(get_num(width) / get_num(height)))
print('desired ratio : {}'.format(ROW_2_WIDTH_COL_3_MM / ROW_2_HEIGHT_MM))

## Row 3

Load data for row 3 (comparisons against other methods)

In [ ]:
# load snpe data
prefix = 'hh_1000_iwloss_svi_cyth_seed2_v3pa01'
dists, infos, losses, nets, posteriors, sims = io.load_prefix(dirs['dir_nets_hh'], prefix)
sim = io.last(sims)
obs_stats = sim.obs
y_obs = sim.obs_trace.reshape(-1,1)
gt = sim.true_params
labels_params = sim.labels_params
param_invtransform = sim.param_invtransform
param_transform = sim.param_transform
gt_transf = param_transform(gt)
n_params = sim.n_params
prior_min = sim.prior_min
prior_max = sim.prior_max
prior_lims = np.concatenate((prior_min.reshape(-1,1),prior_max.reshape(-1,1)),axis=1)
t = sim.t
I = sim.I_obs
A_soma = sim.A_soma
dt = sim.dt
duration = np.max(t)
posterior = io.nth(posteriors, -1)
bm = sim.bm
init = sim.init

# load efree data
prefix_ef = 'hh_1000_svi_cyth_seed1'  # efree
dists_ef, infos_ef, losses_ef, nets_ef, posteriors_ef, sims_ef = io.load_prefix(dirs['dir_nets_hh'], prefix_ef)
l_sims_ef = io.last(sims_ef)
posterior_ef = io.nth(posteriors_ef, 0)

# ibea
prefix_ibea = prefix + '_ibea'
_, halloffame, log, _ = io.load(dirs['dir_genetic_hh'] + prefix_ibea + '.pkl')
best_params_ibea = halloffame[0]
best_sumstats_ibea = best_params_ibea.fitness.values
hlf_ibea = np.asarray(best_params_ibea)

# compute differences between true parameters and modes

# SNPE
m, S = posterior.calc_mean_and_cov()
diff_params = np.abs(np.divide(param_invtransform(m)-gt,gt))
err_params_up_norm = np.divide((param_invtransform(m+np.sqrt(np.diag(S))) - param_invtransform(m)), gt)
err_params_down_norm = np.divide(param_invtransform(m) - param_invtransform(m-np.sqrt(np.diag(S))), gt)

# epsilon-free
m_ef, S_ef = posterior_ef.calc_mean_and_cov()
diff_params_ef = np.abs(np.divide(param_invtransform(m_ef)-gt,gt))
err_params_up_norm_ef = np.divide((param_invtransform(m_ef+np.sqrt(np.diag(S_ef))) - param_invtransform(m_ef)), gt)
err_params_down_norm_ef = np.divide(param_invtransform(m_ef) - param_invtransform(m_ef-np.sqrt(np.diag(S_ef))), gt )

# ibea
diff_params_ibea = np.abs(np.divide(hlf_ibea-gt,gt))
err_params_ibea = np.divide(np.std(halloffame,0) , gt )

# re-order
# sort_indices = np.argsort(diff_params-diff_params_ibea)
sort_indices = np.flip(np.argsort(diff_params_ibea), axis=0)
labels_params_sort = [labels_params[i] for i in sort_indices]

min_m = np.min([np.min(diff_params-err_params_down_norm),
                np.min(diff_params_ef-err_params_down_norm_ef),
                np.min(diff_params_ibea-err_params_ibea)])
max_m = np.max([np.max(diff_params+err_params_up_norm),
                np.max(diff_params_ef+err_params_up_norm_ef),
                np.max(diff_params_ibea+err_params_ibea)])

### Panel G

In [ ]:
fig_inches = (SCALE_IN*mm2inches(1.*ROW_3_WIDTH_COL_1_MM), SCALE_IN*mm2inches(0.98*ROW_3_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC, rc={'text.usetex': False}):
    fig = plt.figure(figsize=fig_inches)

    # histogram of differences
    width = 0.3
    plt.bar(np.linspace(0,n_params-1,n_params),
            diff_params[sort_indices],width, color=COL['SNPE'], align='center',
            yerr=[err_params_down_norm[sort_indices],
                  err_params_up_norm[sort_indices]], ecolor='k', edgecolor = 'none', label='SNPE mean')
    #plt.bar(np.linspace(0,n_params-1,n_params)+2*width,
    #        diff_params_ef[sort_indices],width,color=COL['EFREE'], align='center',
    #        yerr=[err_params_down_norm_ef[sort_indices],
    #              err_params_up_norm_ef[sort_indices]], ecolor='k', edgecolor = 'none', label='epsilon-free mean')
    plt.bar(np.linspace(0,n_params-1,n_params)+1*width,
            diff_params_ibea[sort_indices],width,color=COL['IBEA'], align='center',
            yerr=[err_params_ibea[sort_indices],err_params_ibea[sort_indices]],
            ecolor='k', edgecolor = 'none', label='best ibea')

    ax = plt.gca()
    plt.legend(bbox_to_anchor=(1.15, 1), loc='upper right')
    ax.set_xlim(-1.5*width,n_params+width/2)
    ax.set_xticks(np.linspace(0,n_params-1,n_params)+width/2)
    ax.set_ylim([0, 1])
    ax.set_yticks(np.linspace(0,1,3))
    ax.yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.1f'))
    ax.set_xticklabels(labels_params_sort)
    plt.ylabel(r'||$\hat \theta$ - $\theta^*$|| / $\theta^*}$')

    plt.savefig(PANEL_G, facecolor='None', transparent=True)  # the figure is saved as svg
    plt.close()

svg(PANEL_G)

panel = st.fromfile(PANEL_G)
width, height = panel.get_size()
print('size of svg : {}'.format((width, height)))
print('aspect ratio : {}'.format(get_num(width) / get_num(height)))
print('desired ratio : {}'.format(ROW_1_WIDTH_COL_1_MM / ROW_1_HEIGHT_MM))

## Compose figure

In [ ]:
fig = create_fig(FIG_WIDTH_MM, FIG_HEIGHT_MM)

if False:
    fig = add_grid(fig, 2, 2)
    fig = add_grid(fig, 160/3, 10, font_size_px=0.0001)

yoffset = -1.2
xoffset = -1.
fig = add_svg(fig, PANEL_A, 0 + xoffset, 0 + yoffset)
fig = add_svg(fig, PANEL_B, ROW_1_WIDTH_COL_1_MM + xoffset + 5, 0 + yoffset)
fig = add_svg(fig, PANEL_C, ROW_1_WIDTH_COL_1_MM + ROW_1_WIDTH_COL_2_MM + xoffset, 0 + yoffset)
fig = add_svg(fig, PANEL_D, 0 + xoffset, ROW_2_HEIGHT_MM + yoffset)
fig = add_svg(fig, PANEL_E, ROW_2_WIDTH_COL_1_MM + xoffset + 5, ROW_2_HEIGHT_MM + yoffset - 0.1)
fig = add_svg(fig, PANEL_F, ROW_2_WIDTH_COL_1_MM + ROW_2_WIDTH_COL_2_MM + xoffset + 1., ROW_2_HEIGHT_MM + yoffset)
fig = add_svg(fig, PANEL_G, 0 + xoffset, ROW_1_HEIGHT_MM + ROW_2_HEIGHT_MM + yoffset)

yoffset = 2.3
fig = add_label(fig, 'A', 0, 0 + yoffset)
fig = add_label(fig, 'B', ROW_1_WIDTH_COL_1_MM, 0 + yoffset)
fig = add_label(fig, 'C', ROW_1_WIDTH_COL_1_MM + ROW_1_WIDTH_COL_2_MM, 0 + yoffset)
yoffset = 1.5
fig = add_label(fig, 'D', 0, ROW_2_HEIGHT_MM + yoffset)
fig = add_label(fig, 'E', ROW_2_WIDTH_COL_1_MM, ROW_2_HEIGHT_MM + yoffset)
fig = add_label(fig, 'F', ROW_2_WIDTH_COL_1_MM + ROW_2_WIDTH_COL_2_MM, ROW_2_HEIGHT_MM + yoffset)
yoffset = 1.5
fig = add_label(fig, 'G', 0, ROW_1_HEIGHT_MM + ROW_2_HEIGHT_MM + yoffset)

PATH_SVG = PATH_DROPBOX_FIGS + 'fig3.svg'
fig.save(PATH_SVG)

#svg(PATH_SVG)

!$INKSCAPE --export-pdf $PATH_DROPBOX_FIGS/fig3.pdf $PATH_SVG